# Experiments ECAI24 on the Diabetes dataset

8 input features, all continuous. No known distribution shifts in this dataset, so it is used to mimic the daily model updates.



In [1]:
# Basics
import numpy as np
import pandas as pd
import csv

# sci-kit learn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split,cross_validate
from util_scripts.convert import extract_sklearn_params, custom_nn_model
from joblib import dump, load
import gurobipy

pd.options.display.max_columns = 100
pd.options.display.max_rows = 150

import warnings
warnings.filterwarnings('ignore')

from util_scripts.preprocessor import Preprocessor, min_max_scale
from util_scripts.utilexp import *
from interval import *
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
tf.random.set_seed(1)
np.random.seed(1)

CYAN_COL = '\033[96m'
BLUE_COL = '\033[94m'
RED_COL = '\033[91m'
GREEN_COL = '\033[92m'
YELLOW_COL = '\033[93m'
RESET_COL = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'

In [2]:
df = pd.read_csv("./datasets/diabetes/diabetes.csv")
df = df.dropna()
display(df)
ordinal_features = {}
discrete_features = {}
continuous_features = list(df.columns)[:-1]
#columns = copy.deepcopy(continuous_features)
CLASS = "Outcome"

# min max scale
min_vals = np.min(df[continuous_features], axis=0)
max_vals = np.max(df[continuous_features], axis=0)
df_mm = min_max_scale(df, continuous_features, min_vals, max_vals)
columns = list(df_mm.columns)
# get X, y
X, y = df_mm.drop(columns=['Outcome']), pd.DataFrame(df_mm['Outcome'])

SPLIT = .2
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=SPLIT, shuffle=True,
                                                    random_state=0)
feat_var_map = {}
for i in range(len(X.columns)):
    feat_var_map[i] = [i]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
idx_1 = np.sort(np.random.choice(range(768), 384))
idx_2 = np.array([i for i in range(768) if i not in idx_1])

X1 = pd.DataFrame(data=X.values[idx_1], columns=X.columns)
y1 = pd.DataFrame(data=y.values[idx_1], columns=y.columns)
X2 = pd.DataFrame(data=X.values[idx_2], columns=X.columns)
y2 = pd.DataFrame(data=y.values[idx_2], columns=y.columns)

SPLIT = .2
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, stratify=y1, test_size=SPLIT, shuffle=True,
                                                        random_state=0)


## Train models and Observe model shifts


In [4]:
# Randomdised search + 5-fold cross validation (default)
nn = MLPClassifier(learning_rate='adaptive', random_state=0)

# parameters
max_iter_vals = [int(i) for i in np.linspace(1000, 10000, 10)]
hidden_layer_sizes_vals = [(i) for i in range(5, 16)]
batch_size_vals = [8, 16, 32, 64]
learning_rate_init_vals = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05]

#distributions = dict(max_iter=max_iter_vals, hidden_layer_sizes=hidden_layer_sizes_vals)
distributions = dict(hidden_layer_sizes=hidden_layer_sizes_vals,
                     batch_size=batch_size_vals,
                     learning_rate_init=learning_rate_init_vals,
                     max_iter=max_iter_vals, )

#nns = RandomizedSearchCV(nn, distributions, scoring='f1_macro')
#nns = RandomizedSearchCV(nn, distributions, scoring='accuracy')
#search = nns.fit(X, y)
#print(search.best_params_)

In [5]:
clf = MLPClassifier(learning_rate='adaptive', hidden_layer_sizes=8, learning_rate_init=0.01, batch_size=8,
                    max_iter=7000, random_state=0)

# 5-fold cross validation
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf, X, y, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.2f %s with a std of %0.2f" % (scores[name].mean(), name, scores[name].std()))

clf = MLPClassifier(learning_rate='adaptive', hidden_layer_sizes=8, learning_rate_init=0.01, batch_size=8,
                    max_iter=7000, random_state=0)

clf.fit(X1_train, y1_train)
resres = clf.predict(X1_test.values)
print('\n', classification_report(y1_test, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))
resres = clf.predict(X1.values)
print('\n', classification_report(y1, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))


0.76 test_accuracy with a std of 0.04
0.76 test_precision_macro with a std of 0.04
0.72 test_recall_macro with a std of 0.03
0.73 test_f1_macro with a std of 0.04

                  precision    recall  f1-score   support

 bad credit (0)      0.815     0.880     0.846        50
good credit (1)      0.739     0.630     0.680        27

       accuracy                          0.792        77
      macro avg      0.777     0.755     0.763        77
   weighted avg      0.788     0.792     0.788        77


                  precision    recall  f1-score   support

 bad credit (0)      0.849     0.863     0.856       248
good credit (1)      0.742     0.721     0.731       136

       accuracy                          0.812       384
      macro avg      0.796     0.792     0.794       384
   weighted avg      0.811     0.812     0.812       384



In [6]:
dump(clf, './models/diabetes.joblib')

['./models/diabetes.joblib']

## Computing counterfactuals

#### Procedures

These procedures are covered by UtilExp class

1. Train M on D1
2. Get delta-min, build M+ and M-: incrementally train M 5 times, using different 10% of D2 each time, then get the maximum inf-distance between the incremented models and M. Construct M+ and M- using delta-min
3. Get M2: incrementally train M on D2
4. Select test instances: randomly select 50 D1 instances to explain, clf(x)=0, desired class=1
5. Report metrics using each baseline

#### Metrics
- Proximity: normalised L1: "Scaling Guarantees for Nearest CEs" page 7
- Sparsity: L0
- Validity-delta: percentage of test instances that 1) have counterfactuals valid on m1, 2) counterfactuals valid on M+ and M- under delta_min
- Validity-m2: percentage of test instances that 1) have counterfactual(s), 2) these counterfactual(s) are all valid on both m1 and m2
- LOF: average LOF score

In [7]:
clf = load("./models/diabetes.joblib")
gurobipy.setParam("FeasibilityTol", 1e-03) #1e-09
gurobipy.setParam("OptimalityTol", 1e-03) #1e-09
gurobipy.setParam("IntFeasTol", 1e-03) #1e-05

Restricted license - for non-production use only - expires 2025-11-24
Set parameter FeasibilityTol to value 0.001
Set parameter OptimalityTol to value 0.001
Set parameter IntFeasTol to value 0.001


In [8]:
util_exp = UtilExp(clf, X1, y1, X2, y2, columns, ordinal_features, discrete_features, continuous_features, feat_var_map, num_test_instances=500, gap=0.25)
print(util_exp.delta_max)
print(util_exp.delta_min)

0.3457906217784634
0.1058395646526038


In [9]:
# save model trained on the whole dataset
m2 = copy.deepcopy(clf)
m2.partial_fit(X2, y2)
util_exp.Mmax = m2

In [10]:
# pre-verification on the points soundness 
valids = util_exp.verify_soundness()
print(len(valids))

percentage of sound model changes: 0.27380952380952384
69


In [11]:
valids = util_exp.verify_soundness(update_test_instances=True)

percentage of sound model changes: 0.27380952380952384
test instances updated to sound (x, Delta) pairs, length: 50


Compute the $\delta$ difference between m1 and m2

In [12]:
input_size, n_layers, output_size, output_act, h_act, optimizer, params = extract_sklearn_params(clf)
tf_model = custom_nn_model(input_size, n_layers, params, output_size, h_act, output_act, optimizer)
     
# Set model weights
for k, v in params.items():
	tf_model.layers[k].set_weights(v)
        
tf_model.save('./models/diabetes.h5', save_format='h5')   

input_size, n_layers, output_size, output_act, h_act, optimizer, params = extract_sklearn_params(m2)
tf_model = custom_nn_model(input_size, n_layers, params, output_size, h_act, output_act, optimizer)
     
# Set model weights
for k, v in params.items():
	tf_model.layers[k].set_weights(v)
	
tf_model.save('./models/diabetes_retrained.h5', save_format='h5')  

original_model = tf.keras.models.load_model('./models/diabetes.h5', compile=False)
old_weights = {}
for l in range(1,len(original_model.layers)):
	old_weights[l] = original_model.layers[l].get_weights()


model_retrained = tf.keras.models.load_model('./models/diabetes_retrained.h5', compile=False)
retrained_weights = {}
for l in range(1,len(model_retrained.layers)):
	retrained_weights[l] = model_retrained.layers[l].get_weights()


max_diff = -1
for l in range(1,len(old_weights)):
	old_layer_weights = old_weights[l][0]
	new_retrained_weights = retrained_weights[l][0]

	difference = abs(old_layer_weights - new_retrained_weights)
	
	for list_weights in difference:
		max_distance = max(list_weights)
		if max_distance > max_diff:
			max_diff = max_distance

print("The maximum distance between weights is:", max_diff)

The maximum distance between weights is: 0.27983832


### CFX computation: in the following cells we both compute the CFX using MILP and the proposed probabilistic APΔS approach. 

In [14]:
# OURS-ROBUST: compute CFX based on the n sound points discovered above
ours_robust_ces_apas = util_exp.run_ours_robust(approx=True)
util_exp.evaluate_ces(ours_robust_ces_apas)
cfxs_robust_apas = ours_robust_ces_apas
print(len(cfxs_robust_apas))

50it [00:11,  4.25it/s]

total computation time in s: 11.774839162826538
found: 1.0
average normalised L1: 0.07657846284575133
average normalised L0: 0.3075
average lof score: 1.0
counterfactual validity: 1.0
delta validity: 0.0
m2 validity: 1.0
50


### Compute Robust CFXs using MILP

In [15]:
# OURS-ROBUST: compute CFX based on the n sound points discovered above
ours_robust_ces = util_exp.run_ours_robust()
util_exp.evaluate_ces(ours_robust_ces)
cfxs_robust = ours_robust_ces
print(len(cfxs_robust))

50it [00:06,  8.21it/s]


total computation time in s: 6.095015048980713
found: 1.0
average normalised L1: 0.21184376391537593
average normalised L0: 0.415
average lof score: -0.48
counterfactual validity: 1.0
delta validity: 1.0
m2 validity: 1.0
50


Testing how many robust CFX remains robust after the retraining (i.e., expanding the $\delta$)

In [16]:
# double checking the robustness of the CFXs found by APΔS approach after retraining
tot_robust_cfx = len(cfxs_robust_apas)
robust_cfx_after_retrain = 0

for cfx in cfxs_robust_apas:
    if model_retrained(np.array(cfx.reshape(1,-1))) >= 0.5:
        robust_cfx_after_retrain += 1

print(f"Percentage CFXs robust after the retraing: {(robust_cfx_after_retrain/tot_robust_cfx)*100}%") 

Percentage CFXs robust after the retraing: 100.0%


In [17]:
tot_robust_cfx = len(cfxs_robust)
robust_cfx_after_retrain = 0

for cfx in cfxs_robust:
    if model_retrained(np.array(cfx.reshape(1,-1))) >= 0.5:
        robust_cfx_after_retrain += 1

print(f"Percentage CFXs robust after the retraing: {(robust_cfx_after_retrain/tot_robust_cfx)*100}%") 

Percentage CFXs robust after the retraing: 100.0%


## Experiment: probabilistic approximation approach to compute $\Delta$-robustness

Definition of the algorithms

In [18]:

def estimate_robustness(model, delta, cfx, concretizations, use_biases=True, robustness=True):

    """
    Utility method for the estimation of the CFX (not) Δ-robustness in the INN.

    Returns:
    --------
        rate: float
            estimation of the CFX (not) Δ-robustness computed with 'concretizations' models concretizations from the INN
    """
    np.random.seed(1)

    # Store initial weights
    old_weights = {}
    for l in range(1,len(model.layers)):
        old_weights[l] = model.layers[l].get_weights()

    for _ in range(concretizations):
        
        #perturbated_weights = {}
        input_features = np.array(cfx)

        for l in range(1,len(old_weights)+1):
            layer_weights = old_weights[l][0]
            if use_biases: layer_biases  = old_weights[l][1]
            
            weights_perturbation = np.random.uniform(-delta, delta, layer_weights.shape)
            if use_biases: biases_perturbation = np.random.uniform(-delta, delta, layer_biases.shape)
           
            
            layer_weights = [layer_weights+weights_perturbation]

            if use_biases: 
                layer_biases = [layer_biases+biases_perturbation]
                preactivated_res = np.dot(input_features, layer_weights) + layer_biases
            else:
                preactivated_res = np.dot(input_features, layer_weights)

            if l != len(old_weights):
                #relu
                activated_res = np.maximum(0.0, preactivated_res)
            else:
                #sigmoid
                activated_res = 1/(1 + np.exp(-preactivated_res))
            
            input_features = activated_res
            
        if input_features < 0.5:
            return 0  
    
    return 1

def compute_delta_max_MILP(cfx, delta_init, verbose=False):
  
    lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta_init)))
    if lower < 0.5: return 0 # CFX not robust
        
    delta = delta_init
    while lower >= 0.5: # over-approx lower bound is >= 0.5, i.e., x results robust
        delta = 2*delta
        lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta)))
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Lower is: {lower}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta_new)))
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {lower}')
        
        if lower >= 0.5:
            delta_max = delta_new
        else:
            delta = delta_new



def compute_delta_max(model, cfx, delta_init, concretizations, use_biases=True, verbose=False):
  
    rate = estimate_robustness(model, delta_init, cfx, concretizations, use_biases)
    if rate != 1: return 0 # CFX not robust
        
    delta = delta_init
    while rate == 1: # for all the concretizations x results robust
        delta = 2*delta
        rate = estimate_robustness(model, delta, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Rate is: {rate}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        rate = estimate_robustness(model, delta_new, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {rate}')
        
        if rate == 1:
            delta_max = delta_new
        else:
            delta = delta_new
        

### Start the computation

In [19]:
model = tf.keras.models.load_model('./models/diabetes.h5', compile=False)
alpha = 0.999
R = 0.995
concretizations = int(np.emath.logn(R, (1-alpha)))
delta_init = 0.0001
delta_AAAI = 0.1058395646526038

with open("full_results_diabetes.csv", mode='w', newline='') as file:

    csv_writer = csv.writer(file)
    csv_writer.writerow(["CFX", "AAAI δ_max", "Wilks δ_max", "MILP δ_max", "Difference"])

    wilks = []
    milp = []
    print( f"{CYAN_COL}Condifence α =(1-R^n)={(1-R**concretizations)*100}%, R={R*100}%, Concretizations(n)={concretizations}{RESET_COL}")

    # start computing the new deltas with the approximation
    for cfx in cfxs_robust:

        delta_max_sampling = compute_delta_max(model, cfx.reshape(1,-1), delta_init, concretizations,verbose=False)
        delta_max_MILP = compute_delta_max_MILP(cfx, delta_init,verbose=False) 
        difference = abs(delta_max_sampling-delta_max_MILP)
        wilks.append(delta_max_sampling)
        milp.append(delta_max_MILP)
        csv_writer.writerow([cfx,  delta_AAAI, delta_max_sampling, delta_max_MILP, difference])

        print('δ_max =', delta_max_sampling)
        print('δ improvement w.r.t original MILP\'s δ is',difference)
        print("______________________________________________________________________________________")



    csv_writer.writerow([" "])
    csv_writer.writerow(["Mean MILP", "Mean Wilks"])
    csv_writer.writerow([np.mean(np.array(milp)), np.mean(np.array(wilks))])


Condifence α =(1-R^n)=99.89995272421471%, R=99.5%, Concretizations(n)=1378
δ_max = 0.2855000000000001
δ improvement w.r.t original MILP's δ is 0.1741000000000001
______________________________________________________________________________________
δ_max = 0.28390000000000004
δ improvement w.r.t original MILP's δ is 0.17660000000000003
______________________________________________________________________________________
δ_max = 0.26470000000000005
δ improvement w.r.t original MILP's δ is 0.15570000000000003
______________________________________________________________________________________
δ_max = 0.2743
δ improvement w.r.t original MILP's δ is 0.16549999999999998
______________________________________________________________________________________
δ_max = 0.2569
δ improvement w.r.t original MILP's δ is 0.1484
______________________________________________________________________________________
δ_max = 0.28390000000000004
δ improvement w.r.t original MILP's δ is 0.176600000000000